# Behavioral Gestures

Laban/Bartenieff Movement Analysis (LMA) characterizes gestures as individual movements that convey meaning. Within LMA, behavioral gestures can be seen as functional, everyday, or task-oriented movements. In this notebook, we will employ analytic and geometric methods to devise a rule-based system for identifying common behavioral gestures from extracted features. Since no existing model specifically addresses this task, we will begin with basic behavioral gestures such as *standing*, *sitting*, and *bending of knees*.

We will focus our analysis on [Emma Joubert's Skinny Love (Birdy cover) Contemporary Solo](https://www.youtube.com/watch?v=QOlSCBRmfWY).

https://colab.research.google.com/drive/19txHpN8exWhstO6WVkfmYYVC6uug_oVR#scrollTo=QBrKOeP30RAx
https://colab.research.google.com/drive/1z4IM8kG6ipHN6keadjD-F6vMiIIgViKK
https://colab.research.google.com/drive/1uCuA6We9T5r0WljspEHWPHXCT_2bMKUy

### Required Modules

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cv2

## Behavioral Gestures Segmentation from Static Images

We will selectively detect poses from the frames.

![Image](https://i.ibb.co/zFDftG3/poses.png)

Let's first define the basic positions using using a numerical approach

1. `Standing`: If the difference between the height of the right and left sides is less than 0.08

2. `Sitting down`: If the difference between the y-axis 

3. `Bent Knee`: If the angle from hip to knee not within the range of 85-95


https://colab.research.google.com/drive/1uCuA6We9T5r0WljspEHWPHXCT_2bMKUy#scrollTo=FMJBvXvw-NsF

## Repetition Counter

https://colab.research.google.com/drive/19txHpN8exWhstO6WVkfmYYVC6uug_oVR#scrollTo=TEs_lgNiGv-j

This code estimates positions of different body parts, computes Euclidean distances between them, calculates the difference between heights, difference in x-axis from hip to ankle, and the angle between the hip and knee joint for the left and right legs for each row in the dataframe.

In [5]:
def measure_pose_standing(df, threshold):
    dfs = df.copy()
    dfs['diff_rate_r'] = dfs['diff_height_r_and_l'] / dfs['total_height_r']
    dfs['diff_rate_l'] = dfs['diff_height_r_and_l'] / dfs['total_height_l']
    dfs['both_legs_straight'] = (dfs['diff_rate_r'] <= threshold) & (dfs['diff_rate_l'] <= threshold)
    total = dfs['both_legs_straight'].sum()
    return round(total/24.9,2)

print(f"Estimated standing position {measure_pose_standing(df,0.08)} s")

def measure_pose_sitting(df, threshold):
    dfs = df.copy()
    dfs['diff_rate_r'] = dfs['diff_hip_to_ankle_r']/((dfs['right_hip_x']+dfs['right_ankle_x'])/2)
    dfs['diff_rate_l'] = dfs['diff_hip_to_ankle_r']/((dfs['right_hip_x']+dfs['right_ankle_x'])/2)
    dfs['both_legs_sitting'] = (dfs['diff_rate_r'] <= threshold) & (dfs['diff_rate_l'] <= threshold)
    total = dfs['both_legs_sitting'].sum()
    return round(total/24.9,2)

print(f"Estimated sitting position {measure_pose_sitting(df,0.08)} s")

def measure_pose_bent_knees(df):
    dfs = df[['angle_hip_knee_r','angle_hip_knee_l']].abs()
    dfs['bent_knees'] = (dfs['angle_hip_knee_r'] < 80) | (dfs['angle_hip_knee_r'] > 100) & (dfs['angle_hip_knee_l'] < 80) | (dfs['angle_hip_knee_l'] > 100)
    total = dfs['bent_knees'].sum()
    return round(total/24.9,2)

print(f"Estimated bent knee position {measure_pose_bent_knees(df)} s")

Estimated standing position 23.53 s
Estimated sitting position 37.55 s
Estimated bent knee position 59.36 s


## To be continued